### 1. 按照v1_3 data表的顺序，提取      用户id - 问题绑定话题
### 2. 提取每个用户   回答+关注+感兴趣话题集合
### 3. 给每个用户id - 问题绑定话题打分，附在后面

## 1. 先按照v1_3 data表的顺序，提取      用户id - 问题绑定话题1

In [6]:
import pickle

def save_obj(obj, name ):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [7]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
import logging

log_fmt = "[%(asctime)s] %(levelname)s in %(module)s: %(message)s"
logging.basicConfig(format=log_fmt, level=logging.INFO)

import warnings
warnings.filterwarnings('ignore')


def extract_day(s):
    return s.apply(lambda x: int(x.split('-')[0][1:]))


def extract_hour(s):
    return s.apply(lambda x: int(x.split('-')[1][1:]))

In [8]:
base_path = 'data_set_0926'
# 加载邀请回答数据

train = pd.read_csv(f'{base_path}/invite_info_0926.txt', sep='\t', header=None)
train.columns = ['qid', 'uid', 'dt', 'label']
logging.info("invite %s", train.shape)

test = pd.read_csv(f'{base_path}/invite_info_evaluate_1_0926.txt', sep='\t', header=None)
test.columns = ['qid', 'uid', 'dt']
logging.info("test %s", test.shape)

sub = test.copy()

sub_size = len(sub)

train['day'] = extract_day(train['dt'])
train['hour'] = extract_hour(train['dt'])

test['day'] = extract_day(test['dt'])
test['hour'] = extract_hour(test['dt'])

del train['dt'], test['dt']

[2019-12-15 23:29:35,861] INFO in <ipython-input-8-0e93ec28f02e>: invite (9489162, 4)
[2019-12-15 23:29:37,803] INFO in <ipython-input-8-0e93ec28f02e>: test (1141683, 3)


In [9]:
# 加载问题
ques = pd.read_csv(f'{base_path}/question_info_0926.txt', header=None, sep='\t')
ques.columns = ['qid', 'q_dt', 'title_t1', 'title_t2', 'desc_t1', 'desc_t2', 'topic']
del ques['title_t1'], ques['title_t2'], ques['desc_t1'], ques['desc_t2']
logging.info("ques %s", ques.shape)

ques['q_day'] = extract_day(ques['q_dt'])
ques['q_hour'] = extract_hour(ques['q_dt'])
del ques['q_dt']

[2019-12-15 23:30:18,527] INFO in <ipython-input-9-d1d53554f000>: ques (1829900, 3)


In [10]:
# 加载回答
ans = pd.read_csv(f'{base_path}/answer_info_0926.txt', header=None, sep='\t')
ans.columns = ['aid', 'qid', 'uid', 'ans_dt', 'ans_t1', 'ans_t2', 'is_good', 'is_rec', 'is_dest', 'has_img',
               'has_video', 'word_count', 'reci_cheer', 'reci_uncheer', 'reci_comment', 'reci_mark', 'reci_tks',
               'reci_xxx', 'reci_no_help', 'reci_dis']

del ans['ans_t1'], ans['ans_t2']
logging.info("ans %s", ans.shape)

ans['a_day'] = extract_day(ans['ans_dt'])
ans['a_hour'] = extract_hour(ans['ans_dt'])
del ans['ans_dt']

ans = pd.merge(ans, ques, on='qid')
del ques

[2019-12-15 23:32:03,093] INFO in <ipython-input-10-4f94fbed60c4>: ans (4513735, 18)


In [5]:
# 回答距提问的天数
ans['diff_qa_days'] = ans['a_day'] - ans['q_day']

# 时间窗口划分
# train
# val
train_start = 3838
train_end = 3867

val_start = 3868
val_end = 3874

# 这里标示训练集起始
label_end = 3867
label_start = label_end - 6

#这里标示train_label_feature起始时间
train_label_feature_end = label_end - 7
train_label_feature_start = train_label_feature_end - 22

train_ans_feature_end = label_end - 7
train_ans_feature_start = train_ans_feature_end - 50

val_label_feature_end = val_start - 1
val_label_feature_start = val_label_feature_end - 22

val_ans_feature_end = val_start - 1
val_ans_feature_start = val_ans_feature_end - 50



train_label_feature = train[(train['day'] >= train_label_feature_start) & (train['day'] <= train_label_feature_end)]
logging.info("train_label_feature %s", train_label_feature.shape)

val_label_feature = train[(train['day'] >= val_label_feature_start) & (train['day'] <= val_label_feature_end)]
logging.info("val_label_feature %s", val_label_feature.shape)

train_label = train[(train['day'] > train_label_feature_end)]

logging.info("train feature start %s end %s, label start %s end %s", train_label_feature['day'].min(),
             train_label_feature['day'].max(), train_label['day'].min(), train_label['day'].max())

logging.info("test feature start %s end %s, label start %s end %s", val_label_feature['day'].min(),
             val_label_feature['day'].max(), test['day'].min(), test['day'].max())

# 确定ans的时间范围
# 3807~3874
train_ans_feature = ans[(ans['a_day'] >= train_ans_feature_start) & (ans['a_day'] <= train_ans_feature_end)]

val_ans_feature = ans[(ans['a_day'] >= val_ans_feature_start) & (ans['a_day'] <= val_ans_feature_end)]

logging.info("train ans feature %s, start %s end %s", train_ans_feature.shape, train_ans_feature['a_day'].min(),
             train_ans_feature['a_day'].max())

logging.info("val ans feature %s, start %s end %s", val_ans_feature.shape, val_ans_feature['a_day'].min(),
             val_ans_feature['a_day'].max())

fea_cols = ['is_good', 'is_rec', 'is_dest', 'has_img', 'has_video', 'word_count',
            'reci_cheer', 'reci_uncheer', 'reci_comment', 'reci_mark', 'reci_tks',
            'reci_xxx', 'reci_no_help', 'reci_dis', 'diff_qa_days']

[2019-12-11 10:30:42,383] INFO in <ipython-input-5-fbd8d4d44506>: train_label_feature (6895493, 5)
[2019-12-11 10:30:42,748] INFO in <ipython-input-5-fbd8d4d44506>: val_label_feature (7583553, 5)
[2019-12-11 10:30:42,987] INFO in <ipython-input-5-fbd8d4d44506>: train feature start 3838 end 3860, label start 3861 end 3867
[2019-12-11 10:30:43,010] INFO in <ipython-input-5-fbd8d4d44506>: test feature start 3845 end 3867, label start 3868 end 3874
[2019-12-11 10:30:47,598] INFO in <ipython-input-5-fbd8d4d44506>: train ans feature (3700178, 23), start 3810 end 3860
[2019-12-11 10:30:47,610] INFO in <ipython-input-5-fbd8d4d44506>: val ans feature (3992334, 23), start 3817 end 3867


In [9]:
label_start = label_end - 13
train_label = train[(train['day'] > label_start)]
# pre = train_label.shape
train_label.shape

(4518979, 5)

In [10]:
def extract_feature1(target, label_feature, ans_feature):
    # 问题特征
#     按问题id聚在一起，查看每个问题下对label做'mean', 'sum', 'std', 'count'
#     用法示例在DP_test
    t1 = label_feature.groupby('qid')['label'].agg(['mean', 'sum', 'std', 'count']).reset_index()
    t1.columns = ['qid', 'q_inv_mean', 'q_inv_sum', 'q_inv_std', 'q_inv_count']
    target = pd.merge(target, t1, on='qid', how='left')

    # 用户特征
    t1 = label_feature.groupby('uid')['label'].agg(['mean', 'sum', 'std', 'count']).reset_index()
    t1.columns = ['uid', 'u_inv_mean', 'u_inv_sum', 'u_inv_std', 'u_inv_count']
    target = pd.merge(target, t1, on='uid', how='left')


    # 回答部分特征

    t1 = ans_feature.groupby('qid')['aid'].count().reset_index()
    t1.columns = ['qid', 'q_ans_count']
    target = pd.merge(target, t1, on='qid', how='left')

#     这里不用筛就是0其实
#就你之前遇到的那个问题，标记接受邀请了的但是ans集里找不到他的这条ans 那这里就是0
    t1 = ans_feature.groupby('uid')['aid'].count().reset_index()
    t1.columns = ['uid', 'u_ans_count']
    target = pd.merge(target, t1, on='uid', how='left')

    for col in fea_cols:
        t1 = ans_feature.groupby('uid')[col].agg(['sum', 'max', 'mean']).reset_index()
        t1.columns = ['uid', f'u_{col}_sum', f'u_{col}_max', f'u_{col}_mean']
        target = pd.merge(target, t1, on='uid', how='left')

        t1 = ans_feature.groupby('qid')[col].agg(['sum', 'max', 'mean']).reset_index()
        t1.columns = ['qid', f'q_{col}_sum', f'q_{col}_max', f'q_{col}_mean']
        target = pd.merge(target, t1, on='qid', how='left')
        logging.info("extract %s", col)
    return target


train_label = extract_feature1(train_label,train_label_feature,train_ans_feature)
test = extract_feature1(test, val_label_feature, val_ans_feature)

[2019-12-11 15:15:16,894] INFO in <ipython-input-10-70009d0a7596>: extract is_good
[2019-12-11 15:15:33,523] INFO in <ipython-input-10-70009d0a7596>: extract is_rec
[2019-12-11 15:15:49,862] INFO in <ipython-input-10-70009d0a7596>: extract is_dest
[2019-12-11 15:16:07,649] INFO in <ipython-input-10-70009d0a7596>: extract has_img
[2019-12-11 15:16:25,131] INFO in <ipython-input-10-70009d0a7596>: extract has_video
[2019-12-11 15:16:43,255] INFO in <ipython-input-10-70009d0a7596>: extract word_count
[2019-12-11 15:17:02,361] INFO in <ipython-input-10-70009d0a7596>: extract reci_cheer
[2019-12-11 15:17:22,391] INFO in <ipython-input-10-70009d0a7596>: extract reci_uncheer
[2019-12-11 15:17:42,878] INFO in <ipython-input-10-70009d0a7596>: extract reci_comment
[2019-12-11 15:18:04,166] INFO in <ipython-input-10-70009d0a7596>: extract reci_mark
[2019-12-11 15:18:28,160] INFO in <ipython-input-10-70009d0a7596>: extract reci_tks
[2019-12-11 15:18:53,084] INFO in <ipython-input-10-70009d0a7596>: 

In [11]:
# 特征提取结束
logging.info("train shape %s, test shape %s", train_label.shape, test.shape)
assert len(test) == sub_size

# 加载用户
user = pd.read_csv(f'{base_path}/member_info_0926.txt', header=None, sep='\t')
user.columns = ['uid', 'gender', 'creat_keyword', 'level', 'hot', 'reg_type', 'reg_plat', 'freq', 'uf_b1', 'uf_b2',
                'uf_b3', 'uf_b4', 'uf_b5', 'uf_c1', 'uf_c2', 'uf_c3', 'uf_c4', 'uf_c5', 'score', 'follow_topic',
                'inter_topic']
del user['follow_topic'], user['inter_topic']
logging.info("user %s", user.shape)

unq = user.nunique()
logging.info("user unq %s", unq)

for x in unq[unq == 1].index:
    del user[x]
    logging.info('del unq==1 %s', x)

t = user.dtypes
cats = [x for x in t[t == 'object'].index if x not in ['follow_topic', 'inter_topic', 'uid']]
logging.info("user cat %s", cats)

for d in cats:
    lb = LabelEncoder()
    user[d] = lb.fit_transform(user[d])
    logging.info('encode %s', d)

q_lb = LabelEncoder()
q_lb.fit(list(train_label['qid'].astype(str).values) + list(test['qid'].astype(str).values))
train_label['qid_enc'] = q_lb.transform(train_label['qid'])
test['qid_enc'] = q_lb.transform(test['qid'])

u_lb = LabelEncoder()
u_lb.fit(user['uid'])
train_label['uid_enc'] = u_lb.transform(train_label['uid'])
test['uid_enc'] = u_lb.transform(test['uid'])

# merge user
train_label = pd.merge(train_label, user, on='uid', how='left')
test = pd.merge(test, user, on='uid', how='left')
logging.info("train shape %s, test shape %s", train_label.shape, test.shape)

data = pd.concat((train_label, test), axis=0, sort=True)
del test

[2019-12-11 15:28:20,816] INFO in <ipython-input-11-07381c3d6558>: train shape (4518979, 105), test shape (1141683, 104)
[2019-12-11 15:28:33,865] INFO in <ipython-input-11-07381c3d6558>: user (1931654, 19)
[2019-12-11 15:28:37,782] INFO in <ipython-input-11-07381c3d6558>: user unq uid              1931654
gender                 3
creat_keyword          1
level                  1
hot                    1
reg_type               1
reg_plat               1
freq                   5
uf_b1                  2
uf_b2                  2
uf_b3                  2
uf_b4                  2
uf_b5                  2
uf_c1               2561
uf_c2                291
uf_c3                428
uf_c4               1556
uf_c5                  2
score                732
dtype: int64
[2019-12-11 15:28:37,812] INFO in <ipython-input-11-07381c3d6558>: del unq==1 creat_keyword
[2019-12-11 15:28:37,818] INFO in <ipython-input-11-07381c3d6558>: del unq==1 level
[2019-12-11 15:28:37,819] INFO in <ipython-input-11-0

In [12]:
# count编码
count_fea = ['uid_enc', 'qid_enc', 'gender', 'freq', 'uf_c1', 'uf_c2', 'uf_c3', 'uf_c4', 'uf_c5']
for feat in count_fea:
    col_name = '{}_count'.format(feat)
    data[col_name] = data[feat].map(data[feat].value_counts().astype(int))
    data.loc[data[col_name] < 2, feat] = -1
    data[feat] += 1
    data[col_name] = data[feat].map(data[feat].value_counts().astype(int))
    data[col_name] = (data[col_name] - data[col_name].min()) / (data[col_name].max() - data[col_name].min())
    # 

# 问题被回答的次数


# 压缩数据
t = data.dtypes
for x in t[t == 'int64'].index:
    data[x] = data[x].astype('int32')

for x in t[t == 'float64'].index:
    data[x] = data[x].astype('float32')

data['wk'] = data['day'] % 7

feature_cols = [x for x in data.columns if x not in ('label', 'uid', 'qid', 'dt', 'day')]

In [14]:
# target编码
logging.info("feature size %s", len(feature_cols))

X_train_all = data.iloc[:len(train_label)][feature_cols]
y_train_all = data.iloc[:len(train_label)]['label']
test = data.iloc[len(train_label):]
del data
assert len(test) == sub_size

logging.info("train shape %s, test shape %s", train_label.shape, test.shape)

fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for index, (train_idx, val_idx) in enumerate(fold.split(X=X_train_all, y=y_train_all)):
    break

X_train, X_val, y_train, y_val = X_train_all.iloc[train_idx][feature_cols], X_train_all.iloc[val_idx][feature_cols], \
                                 y_train_all.iloc[train_idx], \
                                 y_train_all.iloc[val_idx]
del train_label, X_train_all

model_lgb = LGBMClassifier(learning_rate=0.1, n_estimators=2000, n_jobs=6, objective='binary', seed=1000, silent=True, device="gpu",gpu_platform_id = 0,gpu_device_id = 3,max_bin=63)

model_lgb.fit(X_train, y_train,
              eval_metric=['logloss', 'auc'],
              eval_set=[(X_val, y_val)],
              early_stopping_rounds=50)

sub['label'] = model_lgb.predict_proba(test[feature_cols])[:, 1]

sub.to_csv('result.txt', index=None, header=None, sep='\t')

[2019-12-11 15:43:58,657] INFO in <ipython-input-14-a6c712d8f443>: feature size 126
[2019-12-11 15:44:00,473] INFO in <ipython-input-14-a6c712d8f443>: train shape (4518979, 120), test shape (1141683, 130)


[1]	valid_0's auc: 0.821561	valid_0's binary_logloss: 0.422829
Training until validation scores don't improve for 50 rounds
[2]	valid_0's auc: 0.828186	valid_0's binary_logloss: 0.406501
[3]	valid_0's auc: 0.836319	valid_0's binary_logloss: 0.393664
[4]	valid_0's auc: 0.8388	valid_0's binary_logloss: 0.383378
[5]	valid_0's auc: 0.840815	valid_0's binary_logloss: 0.374845
[6]	valid_0's auc: 0.842012	valid_0's binary_logloss: 0.367585
[7]	valid_0's auc: 0.843107	valid_0's binary_logloss: 0.361475
[8]	valid_0's auc: 0.844476	valid_0's binary_logloss: 0.356204
[9]	valid_0's auc: 0.845666	valid_0's binary_logloss: 0.351629
[10]	valid_0's auc: 0.846823	valid_0's binary_logloss: 0.347588
[11]	valid_0's auc: 0.847354	valid_0's binary_logloss: 0.344203
[12]	valid_0's auc: 0.848049	valid_0's binary_logloss: 0.341114
[13]	valid_0's auc: 0.84902	valid_0's binary_logloss: 0.338374
[14]	valid_0's auc: 0.849453	valid_0's binary_logloss: 0.336008
[15]	valid_0's auc: 0.850117	valid_0's binary_logloss: 

[129]	valid_0's auc: 0.877287	valid_0's binary_logloss: 0.29531
[130]	valid_0's auc: 0.877329	valid_0's binary_logloss: 0.295267
[131]	valid_0's auc: 0.877425	valid_0's binary_logloss: 0.295167
[132]	valid_0's auc: 0.877507	valid_0's binary_logloss: 0.295074
[133]	valid_0's auc: 0.877583	valid_0's binary_logloss: 0.294983
[134]	valid_0's auc: 0.877659	valid_0's binary_logloss: 0.2949
[135]	valid_0's auc: 0.877713	valid_0's binary_logloss: 0.294848
[136]	valid_0's auc: 0.877751	valid_0's binary_logloss: 0.294807
[137]	valid_0's auc: 0.877804	valid_0's binary_logloss: 0.294752
[138]	valid_0's auc: 0.877874	valid_0's binary_logloss: 0.294684
[139]	valid_0's auc: 0.877972	valid_0's binary_logloss: 0.294583
[140]	valid_0's auc: 0.87803	valid_0's binary_logloss: 0.294518
[141]	valid_0's auc: 0.878126	valid_0's binary_logloss: 0.294395
[142]	valid_0's auc: 0.878219	valid_0's binary_logloss: 0.294301
[143]	valid_0's auc: 0.878324	valid_0's binary_logloss: 0.294195
[144]	valid_0's auc: 0.878381

[256]	valid_0's auc: 0.883173	valid_0's binary_logloss: 0.28891
[257]	valid_0's auc: 0.883185	valid_0's binary_logloss: 0.288897
[258]	valid_0's auc: 0.883192	valid_0's binary_logloss: 0.288891
[259]	valid_0's auc: 0.883198	valid_0's binary_logloss: 0.288885
[260]	valid_0's auc: 0.883198	valid_0's binary_logloss: 0.28888
[261]	valid_0's auc: 0.883241	valid_0's binary_logloss: 0.288839
[262]	valid_0's auc: 0.883251	valid_0's binary_logloss: 0.288828
[263]	valid_0's auc: 0.883319	valid_0's binary_logloss: 0.288754
[264]	valid_0's auc: 0.883339	valid_0's binary_logloss: 0.288733
[265]	valid_0's auc: 0.883361	valid_0's binary_logloss: 0.28871
[266]	valid_0's auc: 0.883391	valid_0's binary_logloss: 0.288677
[267]	valid_0's auc: 0.883394	valid_0's binary_logloss: 0.288673
[268]	valid_0's auc: 0.883415	valid_0's binary_logloss: 0.288647
[269]	valid_0's auc: 0.883431	valid_0's binary_logloss: 0.288621
[270]	valid_0's auc: 0.883441	valid_0's binary_logloss: 0.28861
[271]	valid_0's auc: 0.883448

[383]	valid_0's auc: 0.885439	valid_0's binary_logloss: 0.28635
[384]	valid_0's auc: 0.885449	valid_0's binary_logloss: 0.286336
[385]	valid_0's auc: 0.885497	valid_0's binary_logloss: 0.286276
[386]	valid_0's auc: 0.885516	valid_0's binary_logloss: 0.286254
[387]	valid_0's auc: 0.885537	valid_0's binary_logloss: 0.286219
[388]	valid_0's auc: 0.885553	valid_0's binary_logloss: 0.286196
[389]	valid_0's auc: 0.88556	valid_0's binary_logloss: 0.286188
[390]	valid_0's auc: 0.885577	valid_0's binary_logloss: 0.286169
[391]	valid_0's auc: 0.885588	valid_0's binary_logloss: 0.286155
[392]	valid_0's auc: 0.885593	valid_0's binary_logloss: 0.286149
[393]	valid_0's auc: 0.885594	valid_0's binary_logloss: 0.286148
[394]	valid_0's auc: 0.885627	valid_0's binary_logloss: 0.28611
[395]	valid_0's auc: 0.885638	valid_0's binary_logloss: 0.286097
[396]	valid_0's auc: 0.885656	valid_0's binary_logloss: 0.286076
[397]	valid_0's auc: 0.885685	valid_0's binary_logloss: 0.286037
[398]	valid_0's auc: 0.88573

...

[1399]	valid_0's auc: 0.892212	valid_0's binary_logloss: 0.278301
[1400]	valid_0's auc: 0.892212	valid_0's binary_logloss: 0.278301
[1401]	valid_0's auc: 0.892211	valid_0's binary_logloss: 0.278301
[1402]	valid_0's auc: 0.892212	valid_0's binary_logloss: 0.278301
[1403]	valid_0's auc: 0.892213	valid_0's binary_logloss: 0.2783
[1404]	valid_0's auc: 0.892213	valid_0's binary_logloss: 0.2783
[1405]	valid_0's auc: 0.892224	valid_0's binary_logloss: 0.278285
[1406]	valid_0's auc: 0.89223	valid_0's binary_logloss: 0.278278
[1407]	valid_0's auc: 0.89223	valid_0's binary_logloss: 0.278277
[1408]	valid_0's auc: 0.892234	valid_0's binary_logloss: 0.278273
[1409]	valid_0's auc: 0.892248	valid_0's binary_logloss: 0.278255
[1410]	valid_0's auc: 0.892248	valid_0's binary_logloss: 0.278255
[1411]	valid_0's auc: 0.892263	valid_0's binary_logloss: 0.278236
[1412]	valid_0's auc: 0.892265	valid_0's binary_logloss: 0.278233
[1413]	valid_0's auc: 0.892267	valid_0's binary_logloss: 0.278232
[1414]	valid_0's

[1525]	valid_0's auc: 0.89248	valid_0's binary_logloss: 0.277956
[1526]	valid_0's auc: 0.892481	valid_0's binary_logloss: 0.277953
[1527]	valid_0's auc: 0.892483	valid_0's binary_logloss: 0.277951
[1528]	valid_0's auc: 0.892487	valid_0's binary_logloss: 0.277947
[1529]	valid_0's auc: 0.892488	valid_0's binary_logloss: 0.277945
[1530]	valid_0's auc: 0.892492	valid_0's binary_logloss: 0.277941
[1531]	valid_0's auc: 0.892495	valid_0's binary_logloss: 0.277937
[1532]	valid_0's auc: 0.892495	valid_0's binary_logloss: 0.277937
[1533]	valid_0's auc: 0.892494	valid_0's binary_logloss: 0.277938
[1534]	valid_0's auc: 0.892495	valid_0's binary_logloss: 0.277937
[1535]	valid_0's auc: 0.892495	valid_0's binary_logloss: 0.277937
[1536]	valid_0's auc: 0.892498	valid_0's binary_logloss: 0.277934
[1537]	valid_0's auc: 0.8925	valid_0's binary_logloss: 0.277933
[1538]	valid_0's auc: 0.892505	valid_0's binary_logloss: 0.277928
[1539]	valid_0's auc: 0.892507	valid_0's binary_logloss: 0.277925
[1540]	valid_

[1651]	valid_0's auc: 0.892782	valid_0's binary_logloss: 0.277595
[1652]	valid_0's auc: 0.892783	valid_0's binary_logloss: 0.277593
[1653]	valid_0's auc: 0.892782	valid_0's binary_logloss: 0.277593
[1654]	valid_0's auc: 0.892781	valid_0's binary_logloss: 0.277593
[1655]	valid_0's auc: 0.892784	valid_0's binary_logloss: 0.27759
[1656]	valid_0's auc: 0.892785	valid_0's binary_logloss: 0.277589
[1657]	valid_0's auc: 0.892786	valid_0's binary_logloss: 0.277588
[1658]	valid_0's auc: 0.892789	valid_0's binary_logloss: 0.277586
[1659]	valid_0's auc: 0.892787	valid_0's binary_logloss: 0.277587
[1660]	valid_0's auc: 0.892788	valid_0's binary_logloss: 0.277586
[1661]	valid_0's auc: 0.892789	valid_0's binary_logloss: 0.277585
[1662]	valid_0's auc: 0.892788	valid_0's binary_logloss: 0.277586
[1663]	valid_0's auc: 0.892788	valid_0's binary_logloss: 0.277587
[1664]	valid_0's auc: 0.892789	valid_0's binary_logloss: 0.277586
[1665]	valid_0's auc: 0.892788	valid_0's binary_logloss: 0.277587
[1666]	vali

[1777]	valid_0's auc: 0.893038	valid_0's binary_logloss: 0.277299
[1778]	valid_0's auc: 0.89304	valid_0's binary_logloss: 0.277298
[1779]	valid_0's auc: 0.893039	valid_0's binary_logloss: 0.277299
[1780]	valid_0's auc: 0.893043	valid_0's binary_logloss: 0.277295
[1781]	valid_0's auc: 0.893048	valid_0's binary_logloss: 0.277288
[1782]	valid_0's auc: 0.893051	valid_0's binary_logloss: 0.277284
[1783]	valid_0's auc: 0.893063	valid_0's binary_logloss: 0.277274
[1784]	valid_0's auc: 0.893065	valid_0's binary_logloss: 0.277271
[1785]	valid_0's auc: 0.893077	valid_0's binary_logloss: 0.27726
[1786]	valid_0's auc: 0.893086	valid_0's binary_logloss: 0.277249
[1787]	valid_0's auc: 0.893088	valid_0's binary_logloss: 0.277248
[1788]	valid_0's auc: 0.893088	valid_0's binary_logloss: 0.277247
[1789]	valid_0's auc: 0.89309	valid_0's binary_logloss: 0.277246
[1790]	valid_0's auc: 0.893093	valid_0's binary_logloss: 0.277242
[1791]	valid_0's auc: 0.893098	valid_0's binary_logloss: 0.277237
[1792]	valid_

[1902]	valid_0's auc: 0.893421	valid_0's binary_logloss: 0.27684
[1903]	valid_0's auc: 0.89342	valid_0's binary_logloss: 0.276841
[1904]	valid_0's auc: 0.893421	valid_0's binary_logloss: 0.276842
[1905]	valid_0's auc: 0.89342	valid_0's binary_logloss: 0.276842
[1906]	valid_0's auc: 0.89342	valid_0's binary_logloss: 0.276842
[1907]	valid_0's auc: 0.893419	valid_0's binary_logloss: 0.276842
[1908]	valid_0's auc: 0.893418	valid_0's binary_logloss: 0.276842
[1909]	valid_0's auc: 0.893417	valid_0's binary_logloss: 0.276843
[1910]	valid_0's auc: 0.893419	valid_0's binary_logloss: 0.276842
[1911]	valid_0's auc: 0.89342	valid_0's binary_logloss: 0.276841
[1912]	valid_0's auc: 0.89342	valid_0's binary_logloss: 0.276841
[1913]	valid_0's auc: 0.89342	valid_0's binary_logloss: 0.276842
[1914]	valid_0's auc: 0.89342	valid_0's binary_logloss: 0.276842
[1915]	valid_0's auc: 0.893419	valid_0's binary_logloss: 0.276842
[1916]	valid_0's auc: 0.89342	valid_0's binary_logloss: 0.276841
[1917]	valid_0's au

In [15]:
save_obj(model_lgb,'model.lodel_lgb_01')

In [20]:
model_lgb2 = LGBMClassifier(learning_rate=0.05, n_estimators=2000, n_jobs=6, objective='binary', 
                            seed=1000, silent=True, device="gpu",gpu_platform_id = 0,gpu_device_id = 3,
                            max_bin=63,min_child_samples=64)
model_lgb2.fit(X_train, y_train,
              eval_metric=['logloss', 'auc'],
              eval_set=[(X_val, y_val)],
              early_stopping_rounds=50)

[1]	valid_0's auc: 0.821561	valid_0's binary_logloss: 0.434023
Training until validation scores don't improve for 50 rounds
[2]	valid_0's auc: 0.826396	valid_0's binary_logloss: 0.423995
[3]	valid_0's auc: 0.829593	valid_0's binary_logloss: 0.415429
[4]	valid_0's auc: 0.83031	valid_0's binary_logloss: 0.408018
[5]	valid_0's auc: 0.835289	valid_0's binary_logloss: 0.401274
[6]	valid_0's auc: 0.837163	valid_0's binary_logloss: 0.395224
[7]	valid_0's auc: 0.83903	valid_0's binary_logloss: 0.389775
[15]	valid_0's auc: 0.844114	valid_0's binary_logloss: 0.359869
[16]	valid_0's auc: 0.844603	valid_0's binary_logloss: 0.357287
[17]	valid_0's auc: 0.845157	valid_0's binary_logloss: 0.354901
[18]	valid_0's auc: 0.845827	valid_0's binary_logloss: 0.352578
[19]	valid_0's auc: 0.846307	valid_0's binary_logloss: 0.350478
[20]	valid_0's auc: 0.846644	valid_0's binary_logloss: 0.348525
[21]	valid_0's auc: 0.847055	valid_0's binary_logloss: 0.346688
[22]	valid_0's auc: 0.847631	valid_0's binary_loglos

[136]	valid_0's auc: 0.870144	valid_0's binary_logloss: 0.303285
[137]	valid_0's auc: 0.870206	valid_0's binary_logloss: 0.303218
[138]	valid_0's auc: 0.870319	valid_0's binary_logloss: 0.303096
[139]	valid_0's auc: 0.870421	valid_0's binary_logloss: 0.302989
[140]	valid_0's auc: 0.870504	valid_0's binary_logloss: 0.3029
[141]	valid_0's auc: 0.870582	valid_0's binary_logloss: 0.302798
[142]	valid_0's auc: 0.870684	valid_0's binary_logloss: 0.302683
[143]	valid_0's auc: 0.870747	valid_0's binary_logloss: 0.302617
[144]	valid_0's auc: 0.870857	valid_0's binary_logloss: 0.302502
[145]	valid_0's auc: 0.870927	valid_0's binary_logloss: 0.30243
[146]	valid_0's auc: 0.870991	valid_0's binary_logloss: 0.302364
[147]	valid_0's auc: 0.871082	valid_0's binary_logloss: 0.302256
[148]	valid_0's auc: 0.871164	valid_0's binary_logloss: 0.302172
[149]	valid_0's auc: 0.871239	valid_0's binary_logloss: 0.30209
[150]	valid_0's auc: 0.871321	valid_0's binary_logloss: 0.302008
[151]	valid_0's auc: 0.871376

[263]	valid_0's auc: 0.877773	valid_0's binary_logloss: 0.294947
[264]	valid_0's auc: 0.877813	valid_0's binary_logloss: 0.294896
[265]	valid_0's auc: 0.877847	valid_0's binary_logloss: 0.294857
[266]	valid_0's auc: 0.877887	valid_0's binary_logloss: 0.294821
[267]	valid_0's auc: 0.877922	valid_0's binary_logloss: 0.294785
[268]	valid_0's auc: 0.877964	valid_0's binary_logloss: 0.294742
[269]	valid_0's auc: 0.877993	valid_0's binary_logloss: 0.294707
[270]	valid_0's auc: 0.878022	valid_0's binary_logloss: 0.294677
[271]	valid_0's auc: 0.878054	valid_0's binary_logloss: 0.294641
[272]	valid_0's auc: 0.878077	valid_0's binary_logloss: 0.294613
[273]	valid_0's auc: 0.87811	valid_0's binary_logloss: 0.294569
[274]	valid_0's auc: 0.878155	valid_0's binary_logloss: 0.294517
[275]	valid_0's auc: 0.878182	valid_0's binary_logloss: 0.294488
[276]	valid_0's auc: 0.878245	valid_0's binary_logloss: 0.294416
[277]	valid_0's auc: 0.878294	valid_0's binary_logloss: 0.29437
[278]	valid_0's auc: 0.8783

[401]	valid_0's auc: 0.881384	valid_0's binary_logloss: 0.290983
[402]	valid_0's auc: 0.8814	valid_0's binary_logloss: 0.290967
[403]	valid_0's auc: 0.881448	valid_0's binary_logloss: 0.290912
[404]	valid_0's auc: 0.881472	valid_0's binary_logloss: 0.290886
[405]	valid_0's auc: 0.881497	valid_0's binary_logloss: 0.290862
[406]	valid_0's auc: 0.881525	valid_0's binary_logloss: 0.290827
[407]	valid_0's auc: 0.881535	valid_0's binary_logloss: 0.290816
[408]	valid_0's auc: 0.881551	valid_0's binary_logloss: 0.290799
[409]	valid_0's auc: 0.881567	valid_0's binary_logloss: 0.290778
[410]	valid_0's auc: 0.881603	valid_0's binary_logloss: 0.29074
[411]	valid_0's auc: 0.881626	valid_0's binary_logloss: 0.29071
[412]	valid_0's auc: 0.881642	valid_0's binary_logloss: 0.290695
[413]	valid_0's auc: 0.881659	valid_0's binary_logloss: 0.290675
[414]	valid_0's auc: 0.881672	valid_0's binary_logloss: 0.290661
[415]	valid_0's auc: 0.881723	valid_0's binary_logloss: 0.290605
[416]	valid_0's auc: 0.881739

[541]	valid_0's auc: 0.883789	valid_0's binary_logloss: 0.288342
[542]	valid_0's auc: 0.883797	valid_0's binary_logloss: 0.288333
[543]	valid_0's auc: 0.88381	valid_0's binary_logloss: 0.288313
[544]	valid_0's auc: 0.88382	valid_0's binary_logloss: 0.288297
[545]	valid_0's auc: 0.883837	valid_0's binary_logloss: 0.288279
[546]	valid_0's auc: 0.883849	valid_0's binary_logloss: 0.288268
[547]	valid_0's auc: 0.883862	valid_0's binary_logloss: 0.288252
[548]	valid_0's auc: 0.88387	valid_0's binary_logloss: 0.288244
[549]	valid_0's auc: 0.883872	valid_0's binary_logloss: 0.288241
[550]	valid_0's auc: 0.883875	valid_0's binary_logloss: 0.288238
[551]	valid_0's auc: 0.883886	valid_0's binary_logloss: 0.288225
[552]	valid_0's auc: 0.883889	valid_0's binary_logloss: 0.288222
[553]	valid_0's auc: 0.883898	valid_0's binary_logloss: 0.288212
[554]	valid_0's auc: 0.88391	valid_0's binary_logloss: 0.288199
[555]	valid_0's auc: 0.88392	valid_0's binary_logloss: 0.288177
[556]	valid_0's auc: 0.883943	

...

[1586]	valid_0's auc: 0.890096	valid_0's binary_logloss: 0.280993
[1587]	valid_0's auc: 0.890097	valid_0's binary_logloss: 0.280993
[1588]	valid_0's auc: 0.890101	valid_0's binary_logloss: 0.280989
[1589]	valid_0's auc: 0.890103	valid_0's binary_logloss: 0.280986
[1590]	valid_0's auc: 0.890109	valid_0's binary_logloss: 0.280978
[1591]	valid_0's auc: 0.890111	valid_0's binary_logloss: 0.280976
[1592]	valid_0's auc: 0.890111	valid_0's binary_logloss: 0.280976
[1593]	valid_0's auc: 0.890112	valid_0's binary_logloss: 0.280975
[1594]	valid_0's auc: 0.890113	valid_0's binary_logloss: 0.280974
[1595]	valid_0's auc: 0.890114	valid_0's binary_logloss: 0.280974
[1596]	valid_0's auc: 0.890116	valid_0's binary_logloss: 0.280972
[1597]	valid_0's auc: 0.890118	valid_0's binary_logloss: 0.28097
[1598]	valid_0's auc: 0.890123	valid_0's binary_logloss: 0.280964
[1599]	valid_0's auc: 0.890124	valid_0's binary_logloss: 0.280962
[1600]	valid_0's auc: 0.890125	valid_0's binary_logloss: 0.280961
[1601]	vali

[1711]	valid_0's auc: 0.890458	valid_0's binary_logloss: 0.280575
[1712]	valid_0's auc: 0.89046	valid_0's binary_logloss: 0.28057
[1713]	valid_0's auc: 0.890462	valid_0's binary_logloss: 0.280567
[1714]	valid_0's auc: 0.890465	valid_0's binary_logloss: 0.280564
[1715]	valid_0's auc: 0.890466	valid_0's binary_logloss: 0.280563
[1716]	valid_0's auc: 0.890471	valid_0's binary_logloss: 0.280558
[1717]	valid_0's auc: 0.890485	valid_0's binary_logloss: 0.280541
[1718]	valid_0's auc: 0.890487	valid_0's binary_logloss: 0.28054
[1719]	valid_0's auc: 0.890487	valid_0's binary_logloss: 0.28054
[1720]	valid_0's auc: 0.890495	valid_0's binary_logloss: 0.28053
[1721]	valid_0's auc: 0.890497	valid_0's binary_logloss: 0.280528
[1722]	valid_0's auc: 0.890499	valid_0's binary_logloss: 0.280527
[1723]	valid_0's auc: 0.890503	valid_0's binary_logloss: 0.280522
[1724]	valid_0's auc: 0.890503	valid_0's binary_logloss: 0.280521
[1725]	valid_0's auc: 0.890508	valid_0's binary_logloss: 0.280516
[1726]	valid_0'

[1836]	valid_0's auc: 0.890813	valid_0's binary_logloss: 0.280142
[1837]	valid_0's auc: 0.890813	valid_0's binary_logloss: 0.280142
[1838]	valid_0's auc: 0.890813	valid_0's binary_logloss: 0.280143
[1839]	valid_0's auc: 0.890817	valid_0's binary_logloss: 0.280136
[1840]	valid_0's auc: 0.890818	valid_0's binary_logloss: 0.280134
[1841]	valid_0's auc: 0.890822	valid_0's binary_logloss: 0.280131
[1842]	valid_0's auc: 0.890823	valid_0's binary_logloss: 0.280129
[1843]	valid_0's auc: 0.890825	valid_0's binary_logloss: 0.280127
[1844]	valid_0's auc: 0.890833	valid_0's binary_logloss: 0.280119
[1845]	valid_0's auc: 0.890837	valid_0's binary_logloss: 0.280112
[1846]	valid_0's auc: 0.890837	valid_0's binary_logloss: 0.280112
[1847]	valid_0's auc: 0.890839	valid_0's binary_logloss: 0.280109
[1848]	valid_0's auc: 0.890839	valid_0's binary_logloss: 0.280109
[1849]	valid_0's auc: 0.890844	valid_0's binary_logloss: 0.280104
[1850]	valid_0's auc: 0.890857	valid_0's binary_logloss: 0.280089
[1851]	val

[1961]	valid_0's auc: 0.891221	valid_0's binary_logloss: 0.279643
[1962]	valid_0's auc: 0.891222	valid_0's binary_logloss: 0.279642
[1963]	valid_0's auc: 0.891232	valid_0's binary_logloss: 0.279629
[1964]	valid_0's auc: 0.891233	valid_0's binary_logloss: 0.279629
[1965]	valid_0's auc: 0.891232	valid_0's binary_logloss: 0.279629
[1966]	valid_0's auc: 0.891235	valid_0's binary_logloss: 0.279627
[1967]	valid_0's auc: 0.891234	valid_0's binary_logloss: 0.279627
[1968]	valid_0's auc: 0.891234	valid_0's binary_logloss: 0.279627
[1969]	valid_0's auc: 0.891235	valid_0's binary_logloss: 0.279626
[1970]	valid_0's auc: 0.891237	valid_0's binary_logloss: 0.279624
[1971]	valid_0's auc: 0.891242	valid_0's binary_logloss: 0.279619
[1972]	valid_0's auc: 0.891243	valid_0's binary_logloss: 0.279618
[1973]	valid_0's auc: 0.891244	valid_0's binary_logloss: 0.279617
[1974]	valid_0's auc: 0.891246	valid_0's binary_logloss: 0.279615
[1975]	valid_0's auc: 0.891245	valid_0's binary_logloss: 0.279616
[1976]	val

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               device='gpu', gpu_device_id=3, gpu_platform_id=0,
               importance_type='split', learning_rate=0.05, max_bin=63,
               max_depth=-1, min_child_samples=64, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=2000, n_jobs=6, num_leaves=31,
               objective='binary', random_state=None, reg_alpha=0.0,
               reg_lambda=0.0, seed=1000, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [21]:
sub['label'] = model_lgb2.predict_proba(test[feature_cols])[:, 1]

sub.to_csv('result2.txt', index=None, header=None, sep='\t')

In [22]:
save_obj(model_lgb2,'model.lodel_lgb_02')

In [18]:
model_lgb

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               device='gpu', gpu_device_id=3, gpu_platform_id=0,
               importance_type='split', learning_rate=0.1, max_bin=63,
               max_depth=-1, min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=2000, n_jobs=6, num_leaves=31,
               objective='binary', random_state=None, reg_alpha=0.0,
               reg_lambda=0.0, seed=1000, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)